In [1]:
import xarray as xr
import Rbeast as rb
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/"


Initialization done!


In [2]:
percent = xr.open_dataset(dir + "data/processed_data/percent_cover/percent_cover.nc")[
    "__xarray_dataarray_variable__"
]
changed_pixels_mask = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/changed_pixels.nc"
)

ct = xr.open_dataset(dir + "data/processed_data/confusion_tables/ct_all_years.nc")
pix_id = ct["PIX_INDEX"]
ctn = ct["NORMALIZED_CONFUSION"] * 100
dlcc = ct["DLCC"] * -1  #
conf = ct["CONFUSION"]


Growing season analyses

In [47]:
# arr is an empty matrix
arr = np.arange(0, 448 * 1348).reshape(448, 1348, order="F")
t = pd.date_range(start="1984", end="2014", freq="A-Dec").year  # Time range

lai_growing = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_mean.nc"
)
lai_growing = lai_growing.rename({"latitude": "lat", "longitude": "lon"})

# lai_data = lai.isel(latitude=200,longitude=400).values
lai_changed = lai_growing.where(changed_pixels_mask)

metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
mcmc = rb.args(seed=1)
extra = rb.args(  # a set of options to specify the outputs or computational configurations
    dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
    numThreadsPerCPU=2,  # Paralell  computing: use 2 threads per cpu core
    numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
    printOptions=False,
)

season = "none"
out_growing = rb.beast123(lai_changed.values, metadata, [], mcmc, extra)

cp_growing = out_growing.trend.cp
ncp_med_growing = np.round(out_growing.trend.ncp)

# for i in np.arange(0,ncp_med.shape[0]*ncp_med.shape[1]):
ncp1_growing = np.argwhere(ncp_med_growing == 1)
cp1_growing = np.squeeze(cp_growing[0, :, :])

occ_mat_growing = np.zeros((10, 10, 448 * 1348))
ct_percent_mat_growing = np.zeros((10, 10, 448 * 1348))
ct_percent_mat_growing[:] = np.nan
percent_mat_growing = np.zeros((10, 10, 448 * 1348))
percent_mat_growing[:] = np.nan
ui = np.triu_indices(10, k=1)
li = np.tril_indices(10, k=-1)
no_lcc = 0
counter = 0

for i in np.arange(0, cp1_growing.shape[0]):
    for j in np.arange(0, cp1_growing.shape[1]):
        if np.isnan(cp1_growing[i, j]):
            continue
        cp_time = int(cp1_growing[i, j])
        idx = arr[i, j]
        id = np.where(pix_id.isel(time=0).values == idx)[0]
        ct_sel = ctn.isel(ID=id).sel(time=cp_time).squeeze().values
        np.fill_diagonal(ct_sel, 0)
        I_max = np.unravel_index(ct_sel.argmax(), ct_sel.shape)
        if I_max == (0, 0):
            no_lcc += 1
            continue
        abs_lcc = abs(ct_sel[ui] - ct_sel[li])
        net_lcc = abs_lcc.sum()

        if net_lcc < 2:
            continue
        percent_mat_growing[I_max + (idx,)] = net_lcc
        I_max_reversed = I_max[::-1]
        max_ct = np.argmax(abs_lcc)
        ct_percent_mat_growing[I_max + (idx,)] = max(
            ct_sel[li[0][max_ct], li[1][max_ct]], ct_sel[ui[0][max_ct], ui[1][max_ct]]
        )
        occ_mat_growing[I_max + (idx,)] += 1

        counter = counter + 1

names = [
    "EF",
    "DF",
    "shrub",
    "Herb",
    "sparse",
    "Barren",
    "Fen",
    "Bog",
    "Shallow/Littoral",
    "Water",
]
occurance_growing = occ_mat_growing.sum(axis=2)
df_growing = xr.DataArray(data=occurance_growing, dims= ["LC_t1","LC_t2"],coords={"LC_t1":names,"LC_t2":names})
percent_mat_data_growing = percent_mat_growing[~np.isnan(percent_mat_growing)]
percent_mat_data_df_growing = xr.DataArray(percent_mat_data_growing,dims= ["ID"])
df_growing.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/Rbeast/df_growing.nc")
percent_mat_data_df_growing.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/Rbeast/percent_mat_data_df_growing")

ars1111111111111111x
Parallel computing: thread#1  generated ... 
Parallel computing: thread#2  generated ... 
Parallel computing: thread#3  generated ... 
Parallel computing: thread#4  generated ... 
Parallel computing: thread#5  generated ... 
Parallel computing: thread#6  generated ... 
Parallel computing: thread#7  generated ... 
Parallel computing: thread#8  generated ... 
Parallel computing: thread#9  generated ... 
Parallel computing: thread#10 generated ... 
Parallel computing: thread#11 generated ... 
Parallel computing: thread#12 generated ... 
Parallel computing: thread#13 generated ... 
Parallel computing: thread#14 generated ... 
Parallel computing: thread#15 generated ... 
Parallel computing: thread#16 generated ... 
Parallel computing: thread#17 generated ... 
Parallel computing: thread#18 generated ... 
Parallel computing: thread#19 generated ... 
Parallel computing: thread#20 generated ... 
Parallel computing: thread#21 generated ... 
Parallel computing: thread#22 gene

TypeError: __init__() got an unexpected keyword argument 'index'

Seasonal Analysis

In [3]:
# arr is a matrix
arr = np.arange(0, 448 * 1348).reshape(448, 1348, order="F")
t = pd.date_range(start="1984", end="2014", freq="A-Dec").year  # Time range

lai_monthly = xr.open_dataarray(
    dir
    + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_monthly_resample_mean.nc"
)
lai_monthly = lai_monthly.rename({"latitude": "lat", "longitude": "lon"})


In [4]:
df_tmp_lst = []
percent_mat_data_df_tmp_lst = []
for k in np.arange(5, 11):
    print(k)
    tmp = lai_monthly.where(lai_monthly.time.dt.month == k, drop=True)
    tmp_changed = tmp.where(changed_pixels_mask)
    metadata = rb.args(whichDimIsTime=1, season="none", startTime=1984)
    mcmc = rb.args(seed=2)
    extra = rb.args(  # a set of options to specify the outputs or computational configurations
        dumpInputData=True,  # make a copy of the aggregated input data in the beast ouput
        numThreadsPerCPU=2,  # Paralell  computing: use 2 threads per cpu core
        numParThreads=0,  # `0` means using all CPU cores: total num of ParThreads = numThreadsPerCPU * core Num
        printOptions=False,
        printProgressBar=False,
    )

    season = "none"
    out_tmp = rb.beast123(tmp_changed.values, metadata, [], mcmc, extra)

    cp_tmp = out_tmp.trend.cp
    ncp_med_tmp = np.round(out_tmp.trend.ncp)
    # for i in np.arange(0,ncp_med.shape[0]*ncp_med.shape[1]):
    ncp1_tmp = np.argwhere(ncp_med_tmp == 1)
    cp1_tmp = np.squeeze(cp_tmp[0, :, :])
    occ_mat_tmp = np.zeros((10, 10, 448 * 1348))
    ct_percent_mat_tmp = np.zeros((10, 10, 448 * 1348))
    ct_percent_mat_tmp[:] = np.nan
    percent_mat_tmp = np.zeros((10, 10, 448 * 1348))
    percent_mat_tmp[:] = np.nan
    ui = np.triu_indices(10, k=1)
    li = np.tril_indices(10, k=-1)
    no_lcc = 0
    counter = 0

    for i in np.arange(0, cp1_tmp.shape[0]):
        for j in np.arange(0, cp1_tmp.shape[1]):
            if np.isnan(cp1_tmp[i, j]):
                continue
            cp_time = int(cp1_tmp[i, j])
            idx = arr[i, j]
            id = np.where(pix_id.isel(time=0).values == idx)[0]
            ct_sel = ctn.isel(ID=id).sel(time=cp_time).squeeze().values
            np.fill_diagonal(ct_sel, 0)
            I_max = np.unravel_index(ct_sel.argmax(), ct_sel.shape)
            if I_max == (0, 0):
                no_lcc += 1
                continue
            abs_lcc = abs(ct_sel[ui] - ct_sel[li])
            net_lcc = abs_lcc.sum()

            if net_lcc < 2:
                continue
            percent_mat_tmp[I_max + (idx,)] = net_lcc
            I_max_reversed = I_max[::-1]
            max_ct = np.argmax(abs_lcc)
            ct_percent_mat_tmp[I_max + (idx,)] = max(
                ct_sel[li[0][max_ct], li[1][max_ct]],
                ct_sel[ui[0][max_ct], ui[1][max_ct]],
            )
            occ_mat_tmp[I_max + (idx,)] += 1

            counter = counter + 1

    names = [
        "EF",
        "DF",
        "shrub",
        "Herb",
        "sparse",
        "Barren",
        "Fen",
        "Bog",
        "Shallow/Littoral",
        "Water",
    ]
    occurance_tmp = occ_mat_tmp.sum(axis=2)

    df_tmp_lst.append(
        xr.DataArray(
            data=occurance_tmp,
            dims=["LC_t1", "LC_t2"],
            coords={"LC_t1": names, "LC_t2": names},
        )
    )
    percent_mat_data_tmp = percent_mat_tmp[~np.isnan(percent_mat_tmp)]
    percent_mat_data_df_tmp_lst.append(xr.DataArray(percent_mat_data_tmp, dims="ID"))

    occurance_monthly = xr.concat(df_tmp_lst,dim="months")

    occurance_monthly.to_netcdf(dir+"ata/processed_data/noaa_nc/lai_fapar/Rbeast/occurance_monthly.nc")

5
On the first run, check the CPU instruction set ... 

15
CPU Vendor:
    AMD         = No
    Intel       = Yes
 
OS Features:
    OS AVX      = Yes
    OS AVX512   = No

Hardware Features:
    MMX         = Yes
    x64         = Yes
    ABM         = Yes
    RDRAND      = Yes
    RDSEED      = No
    BMI1        = Yes
    BMI2        = Yes
    ADX         = No
    MPX         = No
    PREFETCHW   = No
    PREFETCHWT1 = No
    RDPID       = No
    GFNI        = No
    VAES        = No

SIMD: 128-bit
    SSE         = Yes
    SSE2        = Yes
    SSE3        = Yes
    SSSE3       = Yes
    SSE4a       = No
    SSE4.1      = Yes
    SSE4.2      = Yes
    AES-NI      = Yes
    SHA         = No

SIMD: 256-bit
    AVX         = Yes
    XOP         = No
    FMA3        = Yes
    FMA4        = No
    AVX2        = Yes

SIMD: 512-bit
    AVX512-F         = No
    AVX512-CD        = No
    AVX512-PF        = No
    AVX512-ER        = No
    AVX512-VL        = No
    AVX512-BW        = No
   